<a href="https://colab.research.google.com/github/vishnudas08/Small-AI-Tool-for-Text-Classification/blob/main/p3_text_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

                              
  #AI Tool for Text Classification Using Transformers


In [1]:
import transformers
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

In [ ]:
#AG NEWS DATASET

In [2]:
train_df= pd.read_csv('/content/train.csv')
test_df= pd.read_csv('/content/test.csv')
train_df.columns
train_df['Class Index'].value_counts()

,count
Class Index,
3,30000
4,30000
2,30000
1,30000


In [20]:
#Data Exploration & Class Balancing

In [3]:
min_samples_per_class = train_df['Class Index'].value_counts().min()
print(min_samples_per_class)
samples_per_class = min(min_samples_per_class, 807)

30000


In [4]:
from sklearn.utils import resample

train_df_reduced = (train_df
                    .groupby('Class Index', group_keys=False)
                    .apply(lambda x: resample(x, replace=False, n_samples=samples_per_class, random_state=42))
                    .reset_index(drop=True))
train_df_reduced

/tmp/ipython-input-1818307666.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: resample(x, replace=False, n_samples=samples_per_class, random_state=42))


,Class Index,Title,Description
0,1,Explosion Rocks Baghdad Neighborhood,"BAGHDAD, Iraq, August 24 -- A car bomb explode..."
1,1,BBC reporters' log,BBC correspondents record events in the Middle...
2,1,Israel welcomes Rice nomination; Palestinians ...,Israel on Tuesday warmly welcomed the naming o...
3,1,Medical Journal Calls for a New Drug Watchdog,Medical researchers said the U.S. needs a syst...
4,1,Militants Kidnap Relatives of Iraqi Minister-TV,Militants have kidnapped two relatives of Iraq...
...,...,...,...
3223,4,Enter your e-mail:,"According to wire reports, Sony has finally de..."
3224,4,Video: Taking It with You (Ziff Davis),Ziff Davis - Easily portable video becomes rea...
3225,4,European Winters Could Disappear by 2080 - Rep...,Reuters - Europe is warming up\more quickly th...
3226,4,Conjunction Junction for the Mars Rovers,In the next few weeks the twin Mars rovers wil...


In [5]:
dict={1:0,2:1,3:2,4:3}
train_df_reduced['Class Index']= train_df_reduced['Class Index'].map(dict)

In [6]:
train_df_reduced['Class Index'].value_counts()

,count
Class Index,
0,807
1,807
2,807
3,807


In [7]:
train_df_reduced['Title']

,Title
0,Explosion Rocks Baghdad Neighborhood
1,BBC reporters' log
2,Israel welcomes Rice nomination; Palestinians ...
3,Medical Journal Calls for a New Drug Watchdog
4,Militants Kidnap Relatives of Iraqi Minister-TV
...,...
3223,Enter your e-mail:
3224,Video: Taking It with You (Ziff Davis)
3225,European Winters Could Disappear by 2080 - Rep...
3226,Conjunction Junction for the Mars Rovers


In [8]:
train_df_reduced= train_df_reduced.drop('Title', axis=1)

In [9]:
train_df_reduced

,Class Index,Description
0,0,"BAGHDAD, Iraq, August 24 -- A car bomb explode..."
1,0,BBC correspondents record events in the Middle...
2,0,Israel on Tuesday warmly welcomed the naming o...
3,0,Medical researchers said the U.S. needs a syst...
4,0,Militants have kidnapped two relatives of Iraq...
...,...,...
3223,3,"According to wire reports, Sony has finally de..."
3224,3,Ziff Davis - Easily portable video becomes rea...
3225,3,Reuters - Europe is warming up\more quickly th...
3226,3,In the next few weeks the twin Mars rovers wil...


In [10]:
train_df_reduced['Class Index'].value_counts()

,count
Class Index,
0,807
1,807
2,807
3,807


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [13]:
print(train_df_reduced)

      Class Index                                        Description
0               0  BAGHDAD, Iraq, August 24 -- A car bomb explode...
1               0  BBC correspondents record events in the Middle...
2               0  Israel on Tuesday warmly welcomed the naming o...
3               0  Medical researchers said the U.S. needs a syst...
4               0  Militants have kidnapped two relatives of Iraq...
...           ...                                                ...
3223            3  According to wire reports, Sony has finally de...
3224            3  Ziff Davis - Easily portable video becomes rea...
3225            3  Reuters - Europe is warming up\more quickly th...
3226            3  In the next few weeks the twin Mars rovers wil...
3227            3  It may be hard to believe in these days of dri...

[3228 rows x 2 columns]


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df_reduced["Description"],train_df_reduced['Class Index'] , test_size=0.2, random_state=42)

In [15]:
len(X_train), len(X_test)

(2582, 646)

In [16]:
num_classes=  train_df_reduced['Class Index'].nunique()

num_classes

4

In [21]:
#Tokenization

In [17]:
from transformers import BertModel, BertTokenizer
from torch.utils.data import DataLoader
tokenizer= BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [18]:
def tokenizer_fn(text,label):
  inputs= tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
  return torch.tensor(inputs['input_ids']), torch.tensor(inputs['attention_mask']), torch.tensor(label, dtype=torch.long)
print(tokenizer_fn(['manu bhaker won the two olympic gold medals'] ,[0,1,2,3]))

(tensor([[  101,  2158,  2226,  1038, 20459,  2121,  2180,  1996,  2048,  4386,
          2751,  6665,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0, 

/tmp/ipython-input-4135886095.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(inputs['input_ids']), torch.tensor(inputs['attention_mask']), torch.tensor(label, dtype=torch.long)


In [19]:
train_input_ids,  train_attention_mask, train_labels  =tokenizer_fn(X_train.values.tolist(), y_train.tolist())
test_inputs_ids, test_attention_mask, test_labels = tokenizer_fn(X_test.values.tolist(), y_test.tolist())


/tmp/ipython-input-4135886095.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(inputs['input_ids']), torch.tensor(inputs['attention_mask']), torch.tensor(label, dtype=torch.long)


In [22]:
print("Train labels:", torch.unique(train_labels))

Train labels: tensor([0, 1, 2, 3])


In [23]:
train_dataset= torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset= torch.utils.data.TensorDataset(test_inputs_ids, test_attention_mask, test_labels)

In [24]:
train_dataLoader=DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataLoader=DataLoader(val_dataset, batch_size=128, shuffle=False)

In [25]:
bert= BertModel.from_pretrained('bert-base-uncased')
bert.config.hidden_size

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

768

In [26]:
class textClassifier(nn.Module):
  def __init__(self, num_classes):
    super(). __init__()
    self.bert= BertModel.from_pretrained('bert-base-uncased')

    for param in self.bert.parameters():
      param.requires_grad=False
    self.classifier= nn.Sequential(
        nn.Linear(self.bert.config.hidden_size, 256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256,num_classes)



    )

  def forward(self,input_ids, attention_mask):
    bert_output= self.bert(input_ids=input_ids,attention_mask=attention_mask)
    sentence_embedding= bert_output.last_hidden_state[:,0,:]
    return self.classifier(sentence_embedding)

In [ ]:
#Training Loop

In [37]:
model= textClassifier(num_classes).to(device)
optimizer= optim.Adam(model.parameters(), lr=0.001)
criterion= nn.CrossEntropyLoss()
#print(model)

In [28]:
epochs=2
for epoch in range(epochs):
  model.train()
  total_train_loss=0

  for batch, (input_ids, attention_mask, labels) in enumerate(train_dataLoader):
    input_ids, attention_mask, labels= input_ids.to(device), attention_mask.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs= model(input_ids, attention_mask).squeeze()
    loss= criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    print(f"Batch {batch}, epoch {epoch}, loss {loss.item():0.2f}")
    total_train_loss+= loss.item()
avg_train_loss= total_train_loss/len(train_dataLoader)
print(f" epoch {epoch +1}/{epochs}, Training loss : {avg_train_loss}")

Batch 0, epoch 0, loss 1.38
Batch 1, epoch 0, loss 1.31
Batch 2, epoch 0, loss 1.26
Batch 3, epoch 0, loss 1.23
Batch 4, epoch 0, loss 1.11
Batch 5, epoch 0, loss 1.08
Batch 6, epoch 0, loss 1.00
Batch 7, epoch 0, loss 0.95
Batch 8, epoch 0, loss 0.87
Batch 9, epoch 0, loss 0.90
Batch 10, epoch 0, loss 0.77
Batch 11, epoch 0, loss 0.71
Batch 12, epoch 0, loss 0.67
Batch 13, epoch 0, loss 0.72
Batch 14, epoch 0, loss 0.61
Batch 15, epoch 0, loss 0.61
Batch 16, epoch 0, loss 0.56
Batch 17, epoch 0, loss 0.56
Batch 18, epoch 0, loss 0.62
Batch 19, epoch 0, loss 0.56
Batch 20, epoch 0, loss 0.63
Batch 0, epoch 1, loss 0.50
Batch 1, epoch 1, loss 0.43
Batch 2, epoch 1, loss 0.60
Batch 3, epoch 1, loss 0.48
Batch 4, epoch 1, loss 0.52
Batch 5, epoch 1, loss 0.45
Batch 6, epoch 1, loss 0.43
Batch 7, epoch 1, loss 0.45
Batch 8, epoch 1, loss 0.37
Batch 9, epoch 1, loss 0.44
Batch 10, epoch 1, loss 0.47
Batch 11, epoch 1, loss 0.42
Batch 12, epoch 1, loss 0.50
Batch 13, epoch 1, loss 0.43
Batch

In [29]:
#Validation & Evaluation

In [31]:
model.eval()
total_val_loss=0
correct_predicted=0
with torch.no_grad():
  for input_ids, attention_mask, labels in val_dataLoader:
    input_ids, attention_mask, labels= input_ids.to(device), attention_mask.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(input_ids, attention_mask).squeeze()
    loss= criterion(outputs, labels)
    total_val_loss +=loss.item()
    preds= outputs.argmax(dim=1)
    correct_predicted=torch.sum(preds == labels)
avg_val_loss= total_val_loss/len(val_dataLoader)
val_accuracy= correct_predicted / len(val_dataset)
print(f" Validation loss : {avg_val_loss} , validation Accuracy {val_accuracy}")

 Validation loss : 0.36536524693171185 , validation Accuracy 0.009287925437092781


In [32]:
label_map = {
    0: 'World',
    1: 'Sports',
    2: 'Business',
    3: 'Sci/Tech'
}

In [33]:
para=(
    """The Indian men's hockey team made history by winning back-to-back Olympic gold medals during its golden era. India clinched its first gold in 1928 at the Amsterdam Olympics, followed by successive victories in 1932 (Los Angeles) and 1936 (Berlin) — a remarkable hat-trick of Olympic golds.
Led by legendary players like Dhyan Chand, the team dominated the field with unmatched skill, speed, and precision. Their performance in 1936 is particularly iconic, where India defeated Germany 8–1 in the final — with Dhyan Chand scoring a hat-trick and silencing a massive Berlin crowd.

India went on to win six consecutive Olympic golds from 1928 to 1956, making it one of the most successful teams in Olympic hockey history."""
)

tesla_para = """
Nikola Tesla conducted groundbreaking experiments in wireless energy transmission during the early 20th century. One of his most ambitious projects was the Wardenclyffe Tower, built on Long Island, New York, which aimed to transmit electricity through the air without wires. Tesla believed that wireless power could provide free energy to the world. Although the project was never completed due to funding issues, it laid the foundation for modern innovations in wireless communication and resonant inductive coupling.
"""
business= """Business Standard All India is a national daily newspaper that offers extensive coverage of business and financial news across India. Established in 1975 and headquartered in New Delhi, it is published by the Business Standard Group, a division of the ABP Group"""

In [34]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [35]:
#Prediction Function

In [38]:
def predict(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, max_length=128)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        predicted_class = torch.argmax(outputs, dim=1).item()
        return label_map[predicted_class]

#print(predict("manu bhaker won the two olympic gold medals"))
print(predict(business))

Business
